In [1]:
import pandas as pd
import nltk
from sqlalchemy import create_engine
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.sentiment import SentimentIntensityAnalyzer

In [2]:
db_connection_string = "postgresql+psycopg2://dauren:_!C#cv+3=aY:EDT@medirec-postgresql.postgres.database.azure.com:5432/postgres"

engine = create_engine(db_connection_string)

In [3]:
def summarize_reviews(reviews_df):
    sia = SentimentIntensityAnalyzer()
    combined_text = " ".join(reviews_df['Review'].values)
    sentences = sent_tokenize(combined_text)
    stop_words = set(stopwords.words('english'))
    words = [word for sentence in sentences for word in sentence.split() if word.lower() not in stop_words]

    sentiments = [sia.polarity_scores(sentence) for sentence in sentences]
    avg_sentiment = {
        'pos': sum(s['pos'] for s in sentiments) / len(sentiments),
        'neu': sum(s['neu'] for s in sentiments) / len(sentiments),
        'neg': sum(s['neg'] for s in sentiments),
    }

    fdist = FreqDist(words)
    keywords = ", ".join([word for word, freq in fdist.most_common(10)])

    sentiment_summary = (
        f"People mostly describe the experience as {'positive' if avg_sentiment['pos'] > avg_sentiment['neg'] else 'negative'}. "
        f"Commonly mentioned words include: {keywords}."
    )
    return sentiment_summary

# Jupyter Kernel Gateway Endpoints

--KernelGatewayApp.allow_origin='*' --KernelGatewayApp.ip='0.0.0.0' --KernelGatewayApp.allow_headers='*' --KernelGatewayApp.allow_credentials=True --KernelGatewayApp.port=8888

In [4]:
# GET /summarize_reviews/:drugId

import json

req = json.loads(REQUEST)
path_params = req["path"]
drug_id = path_params["drugId"]

query = f"""
    SELECT "Review", "Rating"
    FROM "DrugReviews"
    WHERE "DrugId" = '{drug_id}'
    """
reviews_df = pd.read_sql(query, engine)

summary = summarize_reviews(reviews_df)

response = {
    "summary": summary
}

print(json.dumps(response))

NameError: name 'REQUEST' is not defined